# Run A Linear (Simple) Financial Projection

## 1. Initiate Notebook

Import Libraries and set up directories

In [27]:
# Notebook Setup:

# External Libraries:
import sys, os
from pathlib import Path

from IPython.core.display import display
import pandas as pd

%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
notebook_path = Path(os.path.abspath(os.path.join('..')))
modules_path = notebook_path.parent / 'modules'
if modules_path not in sys.path:
    sys.path.insert(0, str(modules_path))

models_path = notebook_path.parent / 'models'
if models_path not in sys.path:
    sys.path.insert(0, str(models_path))

In [29]:
# Internal Libraries:

import modules.distribution
import modules.flux
import models.linear
from modules.space import Apartment
from modules.units import Units
from modules.periodicity import Periodicity

## 2. Run Proforma:

2.1 Initiate Project Parameters:

In [30]:
# 2.1.1: Unit Mix:
unit_mix = {
    Apartment.Type.B1B1B0: 0,
    Apartment.Type.B1B1B1: 1,
    Apartment.Type.B2B1B1: 0,
    Apartment.Type.B2B1B2: 0,
    Apartment.Type.B2B2B0: 0,
    Apartment.Type.B2B2B1: 2,
    Apartment.Type.B3B2B0: 0,
    Apartment.Type.B3B2B1: 1
    }

property_params = {
    'gla': 750.0,
    'sales_price_per_gfa': 6875,
    }

project_params = {
    'start_date': pd.Timestamp('2021-01-01'),
    'periodicity_type': Periodicity.Type.month,
    'preliminaries_duration': 2,
    'construction_duration': 4,
    'sales_duration': 3,
    'sales_fee_rate': 0.02,
    'construction_interest_rate_pa': 0.03,
    'parking_ratio': 1,
    'units': Units.Type.AUD,
    'margin_on_cost_reqd': 0.20
    }
models.linear.compose_phases(project_params)
models.linear.compose_apartments(unit_mix=unit_mix, project_params=project_params)

In [31]:
# Display Project Params in a DataFrame:
project_params_df = pd.DataFrame.from_dict(
    data={key: [str(value)] for key, value in project_params.items()},
    orient='index',
    columns=['project_params'])
display(project_params_df)


,project_params
start_date,2021-01-01 00:00:00
periodicity_type,Type.month
preliminaries_duration,2
construction_duration,4
sales_duration,3
sales_fee_rate,0.02
construction_interest_rate_pa,0.03
parking_ratio,1
units,Type.AUD
margin_on_cost_reqd,0.2


2.2 Compose Preliminaries:

In [32]:
# Prelims:
preliminary_costs_index = {
    'design_planning_engineering_cost': 20000.0,
    'survey_geotech_cost': 5000.0,
    'permitting_inspections_certifications_cost': 10000.0,
    'legal_title_appraisal_cost': 3500.0,
    'taxes_insurance_cost': 1500.0,
    'developer_project_management_cost': 50000.0
    }

preliminaries = models.linear.compose_prelim_costs(
    preliminaries_costs_index=preliminary_costs_index,
    project_params=project_params
)
display("Preliminaries: ")
display(preliminaries.aggregation.transpose())

display("Preliminaries Subtotals: ")
display(preliminaries.collapse().aggregation.transpose())

display("Preliminaries Totals: ")
display(preliminaries.collapse().sum(name='preliminaries_total').to_aggregation(periodicity_type=project_params['periodicity_type']).aggregation.transpose())

'Preliminaries: '

,2021-01-31,2021-02-28
design_planning_engineering,10000.0,10000.0
survey_geotech,2500.0,2500.0
permitting_inspections_certifications,5000.0,5000.0
legal_title_appraisal,1750.0,1750.0
taxes_insurance,750.0,750.0
developer_project_management,25000.0,25000.0


'Preliminaries Subtotals: '

,2021-02-28
design_planning_engineering,20000.0
survey_geotech,5000.0
permitting_inspections_certifications,10000.0
legal_title_appraisal,3500.0
taxes_insurance,1500.0
developer_project_management,50000.0


'Preliminaries Totals: '

,2021-02-28
preliminaries_total,90000.0


2.3 Compose Build Costs:

In [33]:
# Build Costs:
build_costs_index = {
    'construction_cost_shell_pergfa': 1250.0,
    'construction_cost_cores_pergfa': 3000.0,
    'siteworks_cost_pergla': 35.0,
    'parking_cost_per_stall': 15000.0,
    'utilities_cost': 10000.0
}

build_costs = models.linear.compose_build_costs(
    build_costs_index=build_costs_index,
    property_params=property_params,
    project_params=project_params
)

display("Build Costs: ")
display(build_costs.aggregation.transpose())

display("Build Costs Subtotals: ")
display(build_costs.collapse().aggregation.transpose())

display("Build Costs Totals: ")
display(build_costs.collapse().sum(name='build_costs_total').to_aggregation(periodicity_type=project_params['periodicity_type']).aggregation.transpose())

'Build Costs: '

,2021-03-31,2021-04-30,2021-05-31,2021-06-30
construction_shell,4978.515625,54763.671875,141887.695312,116995.117188
construction_cores,117692.038239,210269.149804,110160.501758,13978.310199
siteworks,9638.671875,11689.453125,4511.718750,410.156250
parking,937.500000,10312.500000,26718.750000,22031.250000
utilities,2500.000000,2500.000000,2500.000000,2500.000000


'Build Costs Subtotals: '

,2021-06-30
construction_shell,318625.0
construction_cores,452100.0
siteworks,26250.0
parking,60000.0
utilities,10000.0


'Build Costs Totals: '

,2021-06-30
build_costs_total,866975.0


2.4 Compose Financing Costs:

In [34]:
# Finance Costs:
finance_costs = models.linear.compose_finance_costs(
    project_params=project_params,
    preliminaries_costs=preliminaries,
    build_costs=build_costs
)

display("Financing Costs Calculation: ")
display(finance_costs.aggregation.transpose())#_year.aggregation.transpose())

interest_costs = finance_costs.extract('interest').to_aggregation(periodicity_type=project_params['periodicity_type'])

display("Interest Costs: ")
display(interest_costs.aggregation.transpose())

display("Interest Cost Total: ")
display(interest_costs.collapse().aggregation.transpose())

'Financing Costs Calculation: '

,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
loan_drawdown,45000.0,45000.00000,135746.725739,289534.774804,285778.665820,155914.833637
loan_balance,45112.5,90337.78125,226649.718256,517474.954293,805261.754164,963579.529270
interest,112.5,225.28125,565.211267,1290.461233,2008.134050,2402.941470


'Interest Costs: '

,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
interest,112.5,225.28125,565.211267,1290.461233,2008.13405,2402.94147


'Interest Cost Total: '

,2021-06-30
interest,6604.52927


2.5 Compose Net Development Costs:

In [35]:
net_development_costs = modules.flux.Aggregation.merge(
    aggregations=[preliminaries, build_costs, interest_costs],
    name='net_development_costs',
    periodicity_type=project_params['periodicity_type'])

display("Net Development Costs: ")
display(net_development_costs.aggregation.transpose())

display("Net Development Costs Subtotals: ")
display(net_development_costs.collapse().aggregation.transpose())

display("Net Development Costs Total: ")
display(net_development_costs.collapse().sum(name='net_development_costs_total').to_aggregation(periodicity_type=project_params['periodicity_type']).aggregation.transpose())


'Net Development Costs: '

,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
design_planning_engineering,10000.0,10000.00000,0.000000,0.000000,0.000000,0.000000
survey_geotech,2500.0,2500.00000,0.000000,0.000000,0.000000,0.000000
permitting_inspections_certifications,5000.0,5000.00000,0.000000,0.000000,0.000000,0.000000
legal_title_appraisal,1750.0,1750.00000,0.000000,0.000000,0.000000,0.000000
taxes_insurance,750.0,750.00000,0.000000,0.000000,0.000000,0.000000
developer_project_management,25000.0,25000.00000,0.000000,0.000000,0.000000,0.000000
construction_shell,0.0,0.00000,4978.515625,54763.671875,141887.695312,116995.117188
construction_cores,0.0,0.00000,117692.038239,210269.149804,110160.501758,13978.310199
siteworks,0.0,0.00000,9638.671875,11689.453125,4511.718750,410.156250
parking,0.0,0.00000,937.500000,10312.500000,26718.750000,22031.250000


'Net Development Costs Subtotals: '

,2021-06-30
design_planning_engineering,20000.00000
survey_geotech,5000.00000
permitting_inspections_certifications,10000.00000
legal_title_appraisal,3500.00000
taxes_insurance,1500.00000
developer_project_management,50000.00000
construction_shell,318625.00000
construction_cores,452100.00000
siteworks,26250.00000
parking,60000.00000


'Net Development Costs Total: '

,2021-06-30
net_development_costs_total,963579.52927


3 Compose Revenues:

In [36]:
revenues = models.linear.compose_revenues(
    property_params=property_params,
    project_params=project_params)

display("Revenues: ")
display(revenues.aggregation.transpose())

display("Revenues Subtotals: ")
display(revenues.collapse().aggregation.transpose())

display("Revenues Totals: ")
display(revenues.collapse().sum(name='revenues_total').to_aggregation(periodicity_type=project_params['periodicity_type']).aggregation.transpose())

'Revenues: '

,2021-07-31,2021-08-31,2021-09-30
sales,491452.160494,1.358721e+06,491452.160494
sales_fee,-9829.043210,-2.717441e+04,-9829.043210


'Revenues Subtotals: '

,2021-09-30
sales,2341625.0
sales_fee,-46832.5


'Revenues Totals: '

,2021-09-30
revenues_total,2294792.5


4 Net Development Revenue:

In [37]:
ndr_aggregation = modules.flux.Aggregation(
    name='net_development_revenue',
    affluents=[net_development_costs.sum().invert(), revenues.sum()],
    periodicity_type=project_params['periodicity_type'])

display("Net Development Revenue: ")
display(ndr_aggregation.aggregation.transpose())

net_development_revenue = ndr_aggregation.sum()
display(net_development_revenue.to_aggregation(periodicity_type=project_params['periodicity_type']).aggregation.transpose())
display(net_development_revenue.sum().to_aggregation(periodicity_type=project_params['periodicity_type']).aggregation.transpose())

'Net Development Revenue: '

,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
net_development_costs,-45112.5,-45225.28125,-136311.937006,-290825.236037,-287786.799871,-158317.775106,0.000000,0.000000e+00,0.000000
revenues,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,481623.117284,1.331546e+06,481623.117284


,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
net_development_revenue,-45112.5,-45225.28125,-136311.937006,-290825.236037,-287786.799871,-158317.775106,481623.117284,1.331546e+06,481623.117284


,2021-09-30
net_development_revenue,1.331213e+06


5 Project-level Returns:

In [38]:
margin = (project_params['margin_on_cost_reqd'] * net_development_costs.collapse().sum().movements)[0]
residual_land_value = net_development_revenue.sum().movements[0] - margin
display("Residual Land Value: " + str(residual_land_value))

investment_flow = modules.flux.Flow.from_dict(
    name='investment',
    movements={project_params['start_date']: (-1) * residual_land_value },
    units=project_params['units'])
ndr_aggregation.append(affluents=[investment_flow])

net_development_flow = ndr_aggregation.sum()
display(net_development_flow.to_aggregation(periodicity_type=project_params['periodicity_type']).aggregation.transpose())

irr = net_development_flow.xirr()
display("Project IRR: " + str(irr))



'Residual Land Value: 1138497.0648761098'

,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30
net_development_revenue,-1.183610e+06,-45225.28125,-136311.937006,-290825.236037,-287786.799871,-158317.775106,481623.117284,1.331546e+06,481623.117284


'Project IRR: 0.20993722371668785'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9754a48c-b622-40c5-97bd-64d35f620103' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

